In [2]:
# creating spectrograms from all the files, and saving split labelled versions to disk ready for machine learning
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt

import os
import sys
import cPickle as pickle
import numpy as np

# from data_helpers import load_annotations

annotation_pkl_dir = '/media/michael/Seagate/engage/alison_data/golden_set/extracted/annotations/'
savedir = '/media/michael/Seagate/engage/alison_data/golden_set/extracted/specs/'
base_path = '/media/michael/Seagate/engage/alison_data/golden_set/labels/Golden/'

In [3]:
def gen_spec(x, sr):
    specNStepMod = 0.01    # horizontal resolution of spectogram 0.01
    specNWinMod = 0.03     # vertical resolution of spectogram 0.03

    ## Parameters
    nstep = int(sr * specNStepMod)
    nwin  = int(sr * specNWinMod)
    nfft = nwin

    # Get all windows of x with length n as a single array, using strides to avoid data duplication
    #shape = (nfft, len(range(nfft, len(x), nstep)))
    shape = (nfft, ((x.shape[0] - nfft - 1)/nstep)+1)
    strides = (x.itemsize, nstep*x.itemsize)
    x_wins = np.lib.stride_tricks.as_strided(x, shape=shape, strides=strides)

    # Apply hamming window
    x_wins_ham = np.hamming(x_wins.shape[0])[..., np.newaxis] * x_wins

    # compute fft
    fft_mat = np.fft.fft(x_wins_ham, n=nfft, axis=0)[:(nfft/2), :]

    # log magnitude
    fft_mat_lm = np.log(np.abs(fft_mat))
    fft_mat = np.abs(fft_mat)

    return fft_mat

In [20]:
# create (multi-channel?) spectrogram
files = os.listdir(annotation_pkl_dir)

spec_type = '330'
this_save_dir = savedir + spec_type + '/'
for fname in files:
    if fname.endswith('.pkl'):

        with open(annotation_pkl_dir + fname) as f:
            annots, wav, sample_rate = pickle.load(f)

        spec = gen_spec(wav, sample_rate)

        with open(this_save_dir + fname, 'w') as f:
            pickle.dump(spec, f, -1)
            
        print fname

In [7]:
import librosa

N_FFT = 2048
HOP_LENGTH = 1024 # 512
N_MELS = 32 # 128

# create (multi-channel?) spectrogram
files = os.listdir(annotation_pkl_dir)

spec_type = 'mel'

this_save_dir = savedir + spec_type + '/'
for fname in files:
    if fname.endswith('.pkl'):

        with open(annotation_pkl_dir + fname) as f:
            annots, wav, sample_rate = pickle.load(f)

        spec = librosa.feature.melspectrogram(
            wav, sr=sample_rate, n_fft=N_FFT, hop_length=HOP_LENGTH, n_mels=N_MELS)

        with open(this_save_dir + fname, 'w') as f:
            pickle.dump(spec, f, -1)
            
        print fname

BR20EG-13548_20150720_12000019.pkl
BR28LB-11881_20150801_20300022.pkl
BR28LB-11881_20150803_20300018.pkl
BR4-013378_20150820_22000018.pkl
BR4-013378_20150825_0200003.pkl
BR67US-13534_20150801_12300024.pkl
DA5-011881_20150827_1830007.pkl
E105JP-13548_20131009_0717.pkl
E29RR-013378_20150526_08300012.pkl
E29RR-013378_20150526_18000018.pkl
E29RR-013378_20150530_03000023.pkl
E29RR-013378_20150530_07300012.pkl
E29RR-013378_20150531_05300014.pkl
HA53AA-13548_20130724_0346.pkl
HA53AA-13548_20130727_0954.pkl
HA53AA-13548_20130729_0801.pkl
HA53AA-13548_20130730_0305.pkl
HA86RB-13527_20130726_1319.pkl
BR67US-13534_20150802_07300023.pkl
CM167NP-3527_20150910_07300016.pkl
CM167NP-3527_20150910_2000004.pkl
CR05EF-13527_20130916_0823.pkl
CR05EF-13527_20130921_1236.pkl
CR8-13548_20130918_1109.pkl
CR8-13548_20130921_0806.pkl
HA86RB-13527_20130730_0541.pkl
IG62XL-13534_20150512_21000014.pkl
IG62XL-13534_20150514_17300028.pkl
SE23-13527_20130907_1651.pkl
SE3-13548_20130911_0655.pkl
SW154LA-3527_20130705_

/home/michael/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [8]:
print spec.shape

(32, 1319)
